In [41]:
import os
import json
import sys
sys.path.append("transformers/src")
import csv
from sentence_transformers import SentenceTransformer, util
import random
import datetime
import requests
from tweet_graph import *
import pickle as pkl
from transformers import LongformerModel, LongformerConfig
from transformers import RobertaTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm
import pickle as pkl
from collections import Counter
import math
from deep_rank import *
import warnings
warnings.filterwarnings("ignore")
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [4]:
tweet_map=pkl.load(open("save/tweet_map.pkl","rb"))
user_profile_data=pkl.load(open("save/user_profile_data.pkl","rb"))
en_news_data=pkl.load(open("save/en_news_data.pkl","rb"))
tweet_news_data=pkl.load(open("save/tweet_news_data.pkl","rb"))
edges=pkl.load(open("save/edges.pkl","rb"))

In [5]:
fake_news_list=[]
real_news_list=[]
for e in tqdm(edges):
    #得到下面入节点和出节点的推特id     
    in_node=e.in_node
    out_node=e.out_node
    #尝试得到tweet所引用的新闻
    news_id_in=tweet_news_data.get(in_node,None)
    news_info_in=en_news_data.get(news_id_in,None)
    news_id_out=tweet_news_data.get(out_node,None)
    news_info_out=en_news_data.get(news_id_out,None)
    news_lable_in=news_info_in["label"]
    news_lable_out=news_info_out["label"]
    if news_lable_in=="fake":
        fake_news_list.append(news_id_in)
    else:
        real_news_list.append(news_id_in)
    if news_lable_out=="fake":
        fake_news_list.append(news_id_out)
    else:
        real_news_list.append(news_id_out)
fake_news_list=list(set(fake_news_list))
real_news_list=list(set(real_news_list))
print(len(fake_news_list))
print(len(real_news_list))

100%|██████████| 24957578/24957578 [00:59<00:00, 416930.66it/s]

234
405


In [6]:
#提取有效特征

In [11]:
fake_news_feat={}
real_news_feat={}

for new_id in tqdm(fake_news_list):
    external_affected_degree=0
    internal_affected_degree=0
    for e in (edges):
        #得到下面入节点和出节点的推特id     
        in_node=e.in_node
        out_node=e.out_node
        user_id_in=tweet_map.get(e.in_node,None).get("user_id")
        user_id_out=tweet_map.get(e.out_node,None).get("user_id")
        if user_id_in == user_id_out and user_id_in!=None:
            internal_affected_degree+=e.weight
        if user_id_in != user_id_out and user_id_in!=None:
            external_affected_degree+=e.weight
            
    fake_news_feat[new_id]=[internal_affected_degree,external_affected_degree]
    
for new_id in tqdm(real_news_list):
    external_affected_degree=0
    internal_affected_degree=0
    for e in (edges):
        #得到下面入节点和出节点的推特id     
        in_node=e.in_node
        out_node=e.out_node
        user_id_in=tweet_map.get(e.in_node,None).get("user_id")
        user_id_out=tweet_map.get(e.out_node,None).get("user_id")
        if user_id_in == user_id_out and user_id_in!=None:
            internal_affected_degree+=e.weight
        if user_id_in != user_id_out and user_id_in!=None:
            external_affected_degree+=e.weight
    real_news_feat[new_id]=[internal_affected_degree,external_affected_degree]



100%|██████████| 10/10 [07:01<00:00, 42.19s/it]


In [12]:
#提取文本特征

In [15]:
from liwc import Liwc
liwc = Liwc('LIWC2007.txt')

In [27]:
for news_id,feat in fake_news_feat.items():
    text_feat=np.zeros((100,))
    news_info=en_news_data.get(news_id,None)
    text=news_info["text"]
    parse_rst=liwc.parse(text.split(' '))
    value=[y for x,y in parse_rst.items()]
    text_feat[:len(value)]=value
    fake_news_feat[news_id].extend(text_feat.tolist())

In [29]:
for news_id,feat in real_news_feat.items():
    text_feat=np.zeros((100,))
    news_info=en_news_data.get(news_id,None)
    text=news_info["text"]
    parse_rst=liwc.parse(text.split(' '))
    value=[y for x,y in parse_rst.items()]
    text_feat[:len(value)]=value
    real_news_feat[news_id].extend(text_feat.tolist())

In [37]:
fake_feat=[feat for id_,feat in fake_news_feat.items()]
real_feat=[feat for id_,feat in real_news_feat.items()]
fake_feat=np.array(fake_feat)
real_feat=np.array(real_feat)
all_feat= np.vstack((fake_feat, real_feat))
all_label=np.array([0]*10+[1]*10)
print(all_feat.shape)
print(all_label.shape)

(20, 102)
(20,)


In [34]:
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [38]:
X = all_feat
y = all_label

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 初始化XGBoost分类器
model = xgb.XGBClassifier()

# 训练模型
model.fit(X_train, y_train)

# 预测
y_pred = model.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 83.33%
